# FIX STRUCT DIR, don't stratify by subject. Valid subjects shouldn't be in train set at all.

# Initialization

In [1]:
import os, shutil, importlib
import utils; importlib.reload(utils)

os.environ["CUDA_VISIBLE_DEVICES"]=''
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import h5py
import keras
import sklearn

from glob import glob
from PIL import Image
from os.path import join as jp

from keras import models, optimizers, regularizers, applications
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.layers import *
from keras.optimizers import Adam

from scipy import ndimage

from sklearn import metrics
from sklearn.model_selection import train_test_split

from IPython.display import FileLink

%matplotlib inline
np.set_printoptions(precision=4, linewidth=100)

Using TensorFlow backend.


## Struct Dir

For this comp, need to stratify the valid set based on the drivers as well as the categories

In [9]:
path = '../data/statefarm/'
cats = pd.read_csv('../data/statefarm/driver_imgs_list.csv').set_index('img')

In [10]:
utils.create_dirs(path, cats.classname.unique())

In [21]:
print("Take %d subjects" % np.floor(cats.subject.nunique() * 0.2))

Take 5 subjects


Move random subjects into valid

In [50]:
subj_valid = np.random.choice(cats.subject.unique(), 5, replace=False)
for cl in cats.classname.unique():
    imgs_valid = cats[(cats.classname==cl) & (cats.subject.isin(subj_valid))].index.values
    imgs_train = cats[(cats.classname==cl) & ~(cats.subject.isin(subj_valid))].index.values
    for img in imgs_valid:
        cats.ix[img, 'split'] = 'valid'
        shutil.move(os.path.join(path, 'train', cl, img),
                    os.path.join(path, 'valid', cl))
cats.split.fillna('train', inplace=True)
cats.to_csv(path+'driver_imgs_list.csv')

In [58]:
cats.groupby('split').subject.count()

split
train    18573
valid     3851
Name: subject, dtype: int64

Copy about 100 from each class into sample

In [59]:
for cl in cats.classname.unique():
    cats_train = cats[(cats.classname == cl) & (cats.split == 'train')]
    cats_valid = cats[(cats.classname == cl) & (cats.split == 'valid')]

#     # Remove any subjects that only have 1 occurence in either set
#     cats_valid_cnts = cats_valid.subject.value_counts()
#     cats_train_cnts = cats_train.subject.value_counts()
#     cats_valid_keep = cats_valid_cnts[cats_valid_cnts > 1].index.values
#     cats_train_keep = cats_train_cnts[cats_train_cnts > 1].index.values
#     cats_valid = cats_valid[cats_valid.subject.isin(cats_valid_keep)]    
#     cats_train = cats_train[cats_train.subject.isin(cats_train_keep)]    
    
    # Split data
    _, imgs_train = train_test_split(cats_train.index,
                                     test_size=100,
                                     stratify=cats_train.subject)
    _, imgs_valid = train_test_split(cats_valid.index,
                                     test_size=100,
                                     stratify=cats_valid.subject)
    for img_tr, img_v in zip(imgs_train, imgs_valid):
        shutil.copy(jp(path, 'train', cl, img_tr),
                    jp(path, 'sample', 'train', cl, img_tr))
        shutil.copy(jp(path, 'valid', cl, img_v),
                    jp(path, 'sample', 'valid', cl, img_v))

Copy test data

In [60]:
files = np.random.choice(glob(jp(path, 'test', 'test', '*.jpg')), 100, replace=False)
for img in files:
    shutil.copy(img, jp(path, 'sample', 'test', 'test', img.split('/')[-1]))

# Basic Models - Sample

## Define Paths

In [7]:
# path = '../data/statefarm/'
path = '../data/statefarm/sample/'
path_model = '../data/statefarm/models/'
path_test = path + 'test'
path_train = path + 'train'
path_valid = path + 'valid'
path_img_arrays = path + 'img_arrays/'

path_submit = path_model + 'submissions/'
path_checkpoint = path_model + 'checkpoints/'
path_results = path_model + 'results/'

batch_size = 8

## Create Batches

In [3]:
batch_size = 8

In [4]:
batches_train = utils.get_batches(path+'train', batch_size=batch_size)
batches_valid = utils.get_batches(path+'valid', batch_size=batch_size, shuffle=False)

Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [5]:
classes, filenames = utils.get_classes(path)

Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 100 images belonging to 1 classes.


## Linear Model

In [98]:
model_linear = models.Sequential([
        BatchNormalization(axis=3, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(10, activation='softmax')     
    ])

In [99]:
model_linear.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [100]:
model_linear.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=5, callbacks=None)

Epoch 1/5
1000/1000 [==============================] - 31s - loss: 14.3575 - acc: 0.1030 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/5
1000/1000 [==============================] - 25s - loss: 14.3873 - acc: 0.1060 - val_loss: 9.7031 - val_acc: 0.3980
Epoch 3/5
1000/1000 [==============================] - 25s - loss: 14.4096 - acc: 0.1060 - val_loss: 14.9737 - val_acc: 0.0710
Epoch 4/5
1000/1000 [==============================] - 25s - loss: 14.4096 - acc: 0.1060 - val_loss: 15.8901 - val_acc: 0.0130
Epoch 5/5
1000/1000 [==============================] - 25s - loss: 14.4096 - acc: 0.1060 - val_loss: 16.0386 - val_acc: 0.0040


Model isnt doing anything, lower lr

In [102]:
model_linear = models.Sequential([
        BatchNormalization(axis=1, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(10, activation='softmax')     
    ])
model_linear.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model_linear.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=2, callbacks=None)

Epoch 1/2
1000/1000 [==============================] - 26s - loss: 2.3173 - acc: 0.2490 - val_loss: 1.8748 - val_acc: 0.3430
Epoch 2/2
1000/1000 [==============================] - 25s - loss: 1.3051 - acc: 0.5790 - val_loss: 1.2341 - val_acc: 0.5570


Increase lr 

In [103]:
K.set_value(model_linear.optimizer.lr, 0.0001)

In [105]:
model_linear.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=4, callbacks=None)

Epoch 1/4
1000/1000 [==============================] - 26s - loss: 3.0042 - acc: 0.5600 - val_loss: 1.8723 - val_acc: 0.6950
Epoch 2/4
1000/1000 [==============================] - 25s - loss: 1.4681 - acc: 0.7440 - val_loss: 1.9966 - val_acc: 0.7310
Epoch 3/4
1000/1000 [==============================] - 25s - loss: 0.8713 - acc: 0.8410 - val_loss: 1.0877 - val_acc: 0.8224
Epoch 4/4
1000/1000 [==============================] - 25s - loss: 0.4035 - acc: 0.8980 - val_loss: 0.8628 - val_acc: 0.8250


acc_valid stablizes around 82. Generate random valid batches to see if there is sampling bias

In [106]:
batches_rand = utils.get_batches(jp(path, 'valid'), batch_size=batch_size, shuffle=True)

Found 1000 images belonging to 10 classes.


In [115]:
val_scores = [model_linear.evaluate_generator(batches_rand, batches_rand.n/10)
              for i in range(10)]

In [116]:
np.round(val_scores, 2)

array([[ 1.21,  0.79],
       [ 0.89,  0.84],
       [ 0.83,  0.83],
       [ 1.09,  0.82],
       [ 1.14,  0.81],
       [ 1.06,  0.79],
       [ 1.57,  0.78],
       [ 0.91,  0.77],
       [ 0.81,  0.81],
       [ 1.12,  0.81]])

## Linear Model with Maxout

In [117]:
model_linear_mo = models.Sequential([
        BatchNormalization(axis=1, input_shape=(224, 224, 3)),
        Flatten(),
        MaxoutDense(10),
        Activation('softmax')
    ])
model_linear_mo.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model_linear_mo.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=2, callbacks=None)

Epoch 1/2
1000/1000 [==============================] - 26s - loss: 6.7963 - acc: 0.1230 - val_loss: 5.0418 - val_acc: 0.2570
Epoch 2/2
1000/1000 [==============================] - 25s - loss: 3.8167 - acc: 0.2760 - val_loss: 3.2944 - val_acc: 0.3460


In [119]:
K.set_value(model_linear.optimizer.lr, 0.0001)

In [121]:
model_linear_mo.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=5, callbacks=None)

Epoch 1/5
1000/1000 [==============================] - 25s - loss: 1.5415 - acc: 0.5800 - val_loss: 2.7848 - val_acc: 0.4222
Epoch 2/5
1000/1000 [==============================] - 24s - loss: 0.9965 - acc: 0.7110 - val_loss: 1.8750 - val_acc: 0.5000
Epoch 3/5
1000/1000 [==============================] - 24s - loss: 0.6589 - acc: 0.8110 - val_loss: 1.8518 - val_acc: 0.5450
Epoch 4/5
1000/1000 [==============================] - 25s - loss: 0.4296 - acc: 0.8770 - val_loss: 1.4340 - val_acc: 0.6210
Epoch 5/5
1000/1000 [==============================] - 25s - loss: 0.2806 - acc: 0.9370 - val_loss: 1.6638 - val_acc: 0.6220


## L2 regularization

In [128]:
model_linear_l2 = models.Sequential([
        BatchNormalization(axis=3, input_shape=(224, 224, 3)),
        Flatten(),
        Dense(10, activation='softmax', W_regularizer=regularizers.l2(l=0.01))     
    ])
model_linear_l2.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_linear_l2.fit_generator(batches_train, samples_per_epoch=batches_train.n,
                           validation_data=baches_valid, nb_val_samples=batches_valid.n,
                           nb_epoch=5, callbacks=None)

Epoch 1/5
1000/1000 [==============================] - 26s - loss: 10.3370 - acc: 0.1920 - val_loss: 11.0494 - val_acc: 0.2330
Epoch 2/5
1000/1000 [==============================] - 25s - loss: 9.5793 - acc: 0.2880 - val_loss: 10.3611 - val_acc: 0.3430
Epoch 3/5
1000/1000 [==============================] - 25s - loss: 9.3116 - acc: 0.3430 - val_loss: 10.8533 - val_acc: 0.1856
Epoch 4/5
1000/1000 [==============================] - 25s - loss: 6.9767 - acc: 0.4680 - val_loss: 7.9728 - val_acc: 0.4760
Epoch 5/5
1000/1000 [==============================] - 25s - loss: 5.8017 - acc: 0.5820 - val_loss: 1.4761 - val_acc: 0.7570


## Single Conv Layer

In [10]:
def gen_model_conv(lr):
    model_conv = models.Sequential([
            BatchNormalization(axis=3, input_shape=(224, 224, 3)),
            Convolution2D(32, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Convolution2D(64, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Flatten(),
            MaxoutDense(100),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])
    model_conv.compile(Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model_conv

def test_model_conv(batches, epochs2=4):
    model_conv = models.Sequential([
            BatchNormalization(axis=3, input_shape=(224, 224, 3)),
            Convolution2D(32, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Convolution2D(64, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Flatten(),
            MaxoutDense(100),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])
    model_conv.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model_conv.fit_generator(batches, samples_per_epoch=batches.n,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n,
                         nb_epoch=2, callbacks=None)
    K.set_value(model_conv.optimizer.lr, 1e-3)
    model_conv.fit_generator(batches, samples_per_epoch=batches.n,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n,
                         nb_epoch=epochs2, callbacks=None)
    return model_conv

In [13]:
model_conv = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 57s - loss: 1.9145 - acc: 0.3880 - val_loss: 3.1522 - val_acc: 0.1220
Epoch 2/2
1000/1000 [==============================] - 55s - loss: 0.4965 - acc: 0.9000 - val_loss: 2.8061 - val_acc: 0.1890
Epoch 1/4
1000/1000 [==============================] - 60s - loss: 1.1152 - acc: 0.6550 - val_loss: 1.1022 - val_acc: 0.5590
Epoch 2/4
1000/1000 [==============================] - 61s - loss: 0.2055 - acc: 0.9650 - val_loss: 0.7434 - val_acc: 0.7510
Epoch 3/4
1000/1000 [==============================] - 62s - loss: 0.0418 - acc: 0.9980 - val_loss: 0.3475 - val_acc: 0.9060
Epoch 4/4
1000/1000 [==============================] - 64s - loss: 0.0155 - acc: 0.9980 - val_loss: 0.2117 - val_acc: 0.9480


Conv model trains very quickly. Add regularization

## Data Augmentation

Test different data augmentations to find best

#### Width

In [11]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [12]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 57s - loss: 2.5272 - acc: 0.2260 - val_loss: 3.1439 - val_acc: 0.1490
Epoch 2/2
1000/1000 [==============================] - 58s - loss: 1.5965 - acc: 0.4640 - val_loss: 2.4761 - val_acc: 0.3200
Epoch 1/4
1000/1000 [==============================] - 59s - loss: 1.9834 - acc: 0.3580 - val_loss: 1.3972 - val_acc: 0.5340
Epoch 2/4
1000/1000 [==============================] - 59s - loss: 1.2520 - acc: 0.5810 - val_loss: 1.2096 - val_acc: 0.6000
Epoch 3/4
1000/1000 [==============================] - 60s - loss: 0.9778 - acc: 0.6900 - val_loss: 0.8568 - val_acc: 0.7090
Epoch 4/4
1000/1000 [==============================] - 60s - loss: 0.7639 - acc: 0.7580 - val_loss: 0.7602 - val_acc: 0.7580


#### Height

In [13]:
gen_t = image.ImageDataGenerator(height_shift_range=0.1, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [14]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 55s - loss: 2.4805 - acc: 0.2170 - val_loss: 3.3178 - val_acc: 0.2510
Epoch 2/2
 504/1000 [==============>...............] - ETA: 22s - loss: 1.6205 - acc: 0.4663

KeyboardInterrupt: 

#### Rotation

In [9]:
gen_t = image.ImageDataGenerator(rotation_range=15, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [ ]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 68s - loss: 2.4955 - acc: 0.2280 - val_loss: 3.2073 - val_acc: 0.1690
Epoch 2/2
1000/1000 [==============================] - 54s - loss: 1.5611 - acc: 0.4870 - val_loss: 2.5235 - val_acc: 0.3040
Epoch 1/4
1000/1000 [==============================] - 55s - loss: 1.8570 - acc: 0.4140 - val_loss: 1.3541 - val_acc: 0.5740
Epoch 2/4
1000/1000 [==============================] - 55s - loss: 1.0812 - acc: 0.6450 - val_loss: 0.9412 - val_acc: 0.6930
Epoch 3/4
1000/1000 [==============================] - 55s - loss: 0.7232 - acc: 0.7860 - val_loss: 0.6118 - val_acc: 0.8180
Epoch 4/4
 512/1000 [==============>...............] - ETA: 21s - loss: 0.6679 - acc: 0.7695

#### Shearing

In [9]:
gen_t = image.ImageDataGenerator(shear_range=0.1, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [ ]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 68s - loss: 2.4955 - acc: 0.2280 - val_loss: 3.2073 - val_acc: 0.1690
Epoch 2/2
1000/1000 [==============================] - 54s - loss: 1.5611 - acc: 0.4870 - val_loss: 2.5235 - val_acc: 0.3040
Epoch 1/4
1000/1000 [==============================] - 55s - loss: 1.8570 - acc: 0.4140 - val_loss: 1.3541 - val_acc: 0.5740
Epoch 2/4
1000/1000 [==============================] - 55s - loss: 1.0812 - acc: 0.6450 - val_loss: 0.9412 - val_acc: 0.6930
Epoch 3/4
1000/1000 [==============================] - 55s - loss: 0.7232 - acc: 0.7860 - val_loss: 0.6118 - val_acc: 0.8180
Epoch 4/4
 512/1000 [==============>...............] - ETA: 21s - loss: 0.6679 - acc: 0.7695

#### Zoom

In [9]:
gen_t = image.ImageDataGenerator(zoom_range=0.1, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [ ]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 68s - loss: 2.4955 - acc: 0.2280 - val_loss: 3.2073 - val_acc: 0.1690
Epoch 2/2
1000/1000 [==============================] - 54s - loss: 1.5611 - acc: 0.4870 - val_loss: 2.5235 - val_acc: 0.3040
Epoch 1/4
1000/1000 [==============================] - 55s - loss: 1.8570 - acc: 0.4140 - val_loss: 1.3541 - val_acc: 0.5740
Epoch 2/4
1000/1000 [==============================] - 55s - loss: 1.0812 - acc: 0.6450 - val_loss: 0.9412 - val_acc: 0.6930
Epoch 3/4
1000/1000 [==============================] - 55s - loss: 0.7232 - acc: 0.7860 - val_loss: 0.6118 - val_acc: 0.8180
Epoch 4/4
 512/1000 [==============>...............] - ETA: 21s - loss: 0.6679 - acc: 0.7695

#### Channel Shift

In [9]:
gen_t = image.ImageDataGenerator(channel_shift_range=10, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [ ]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 68s - loss: 2.4955 - acc: 0.2280 - val_loss: 3.2073 - val_acc: 0.1690
Epoch 2/2
1000/1000 [==============================] - 54s - loss: 1.5611 - acc: 0.4870 - val_loss: 2.5235 - val_acc: 0.3040
Epoch 1/4
1000/1000 [==============================] - 55s - loss: 1.8570 - acc: 0.4140 - val_loss: 1.3541 - val_acc: 0.5740
Epoch 2/4
1000/1000 [==============================] - 55s - loss: 1.0812 - acc: 0.6450 - val_loss: 0.9412 - val_acc: 0.6930
Epoch 3/4
1000/1000 [==============================] - 55s - loss: 0.7232 - acc: 0.7860 - val_loss: 0.6118 - val_acc: 0.8180
Epoch 4/4
 512/1000 [==============>...............] - ETA: 21s - loss: 0.6679 - acc: 0.7695

#### All

In [11]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1,
                                 rotation_range=15, shear_range=0.1,
                                 zoom_range=0.1, channel_shift_range=10)
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [14]:
model_conv = gen_model_conv(lr=0.001)
model_conv.fit_generator(batches_train, batches_train.n, nb_epoch=4,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n)

Epoch 1/4
1000/1000 [==============================] - 56s - loss: 2.5873 - acc: 0.1520 - val_loss: 2.1937 - val_acc: 0.1230
Epoch 2/4
1000/1000 [==============================] - 59s - loss: 2.2352 - acc: 0.2240 - val_loss: 2.0247 - val_acc: 0.2690
Epoch 3/4
1000/1000 [==============================] - 66s - loss: 2.1095 - acc: 0.2590 - val_loss: 2.1535 - val_acc: 0.2390
Epoch 4/4
1000/1000 [==============================] - 76s - loss: 2.0648 - acc: 0.2690 - val_loss: 1.8062 - val_acc: 0.3400


In [15]:
model_conv.fit_generator(batches_train, batches_train.n, nb_epoch=20,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n)

Epoch 1/20
1000/1000 [==============================] - 58s - loss: 2.0057 - acc: 0.2810 - val_loss: 1.6103 - val_acc: 0.4020
Epoch 2/20
1000/1000 [==============================] - 61s - loss: 1.9216 - acc: 0.3340 - val_loss: 1.5911 - val_acc: 0.4570
Epoch 3/20
1000/1000 [==============================] - 63s - loss: 1.8867 - acc: 0.3430 - val_loss: 1.4882 - val_acc: 0.4620
Epoch 4/20
1000/1000 [==============================] - 71s - loss: 1.8229 - acc: 0.3380 - val_loss: 1.3811 - val_acc: 0.5660
Epoch 5/20
1000/1000 [==============================] - 69s - loss: 1.7744 - acc: 0.3850 - val_loss: 1.4230 - val_acc: 0.5040
Epoch 6/20
1000/1000 [==============================] - 85s - loss: 1.6492 - acc: 0.4280 - val_loss: 1.2146 - val_acc: 0.6100
Epoch 7/20
1000/1000 [==============================] - 78s - loss: 1.6039 - acc: 0.4670 - val_loss: 1.1731 - val_acc: 0.6010
Epoch 8/20
1000/1000 [==============================] - 75s - loss: 1.5430 - acc: 0.4730 - val_loss: 1.0914 - val_acc:

# Basic Models - Full

Do previous experiments on full dataset

## Define Paths

In [2]:
# path = '../data/statefarm/'
path = '../data/statefarm/sample/'
path_model = '../data/statefarm/models/'
path_test = path + 'test'
path_train = path + 'train'
path_valid = path + 'valid'
path_img_arrays = path + 'img_arrays/'

path_submit = path_model + 'submissions/'
path_checkpoint = path_model + 'checkpoints/'
path_results = path_model + 'results/'

## Create Batches

In [3]:
batch_size = 64

In [4]:
batches_train = utils.get_batches(path+'train', batch_size=batch_size)
batches_valid = utils.get_batches(path+'valid', batch_size=batch_size, shuffle=False)


Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


Convert data to bcolz arrays

In [18]:
# utils.compress_imgs(path_train, path_img_arrays+'train.bc', chunk_size=5000)
# utils.compress_imgs(path_valid, path_img_arrays+'valid.bc')
# utils.compress_imgs(path_test, path_img_arrays+'test.bc', chunk_size=7500)

Load data from arrays

In [20]:
data_train = utils.load_array_bcolz(path_img_arrays+'train.bc')
data_valid = utils.load_array_bcolz(path_img_arrays+'valid.bc')
# data_test = utils.load_array_bcolz(path_img_arrays+'test.bc')
classes, filenames = utils.get_classes(path)
labels_train = classes[2]
labels_valid = classes[3]

FileNotFoundError: [Errno 2] No such file or directory: '../data/statefarm/img_arrays/test.bc/meta/sizes'

## Single Conv Layer

In [13]:
def gen_model_conv(lr):
    model_conv = models.Sequential([
            BatchNormalization(axis=3, input_shape=(224, 224, 3)),
            Convolution2D(32, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Convolution2D(64, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Flatten(),
            MaxoutDense(100),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])
    model_conv.compile(Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model_conv

def test_model_conv(batches, epochs2=4):
    model_conv = models.Sequential([
            BatchNormalization(axis=3, input_shape=(224, 224, 3)),
            Convolution2D(32, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Convolution2D(64, 3, 3, activation='relu'),
            BatchNormalization(axis=3),
            MaxPooling2D((3, 3)),
            Flatten(),
            MaxoutDense(100),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])
    model_conv.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model_conv.fit_generator(batches, samples_per_epoch=batches.n,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n,
                         nb_epoch=2, callbacks=None)
    K.set_value(model_conv.optimizer.lr, 1e-3)
    model_conv.fit_generator(batches, samples_per_epoch=batches.n,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n,
                         nb_epoch=epochs2, callbacks=None)
    return model_conv

### No Data Aug

In [14]:
model_conv = test_model_conv(batches_train)

Epoch 1/2
17934/17934 [==============================] - 299s - loss: 0.3415 - acc: 0.9012 - val_loss: 0.0621 - val_acc: 0.9844

KeyboardInterrupt: 

### Data Aug

#### Width

In [11]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [12]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 57s - loss: 2.5272 - acc: 0.2260 - val_loss: 3.1439 - val_acc: 0.1490
Epoch 2/2
1000/1000 [==============================] - 58s - loss: 1.5965 - acc: 0.4640 - val_loss: 2.4761 - val_acc: 0.3200
Epoch 1/4
1000/1000 [==============================] - 59s - loss: 1.9834 - acc: 0.3580 - val_loss: 1.3972 - val_acc: 0.5340
Epoch 2/4
1000/1000 [==============================] - 59s - loss: 1.2520 - acc: 0.5810 - val_loss: 1.2096 - val_acc: 0.6000
Epoch 3/4
1000/1000 [==============================] - 60s - loss: 0.9778 - acc: 0.6900 - val_loss: 0.8568 - val_acc: 0.7090
Epoch 4/4
1000/1000 [==============================] - 60s - loss: 0.7639 - acc: 0.7580 - val_loss: 0.7602 - val_acc: 0.7580


#### Height

In [13]:
gen_t = image.ImageDataGenerator(height_shift_range=0.1, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [14]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 55s - loss: 2.4805 - acc: 0.2170 - val_loss: 3.3178 - val_acc: 0.2510
Epoch 2/2
 504/1000 [==============>...............] - ETA: 22s - loss: 1.6205 - acc: 0.4663

KeyboardInterrupt: 

#### Rotation

In [9]:
gen_t = image.ImageDataGenerator(rotation_range=15, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [ ]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 68s - loss: 2.4955 - acc: 0.2280 - val_loss: 3.2073 - val_acc: 0.1690
Epoch 2/2
1000/1000 [==============================] - 54s - loss: 1.5611 - acc: 0.4870 - val_loss: 2.5235 - val_acc: 0.3040
Epoch 1/4
1000/1000 [==============================] - 55s - loss: 1.8570 - acc: 0.4140 - val_loss: 1.3541 - val_acc: 0.5740
Epoch 2/4
1000/1000 [==============================] - 55s - loss: 1.0812 - acc: 0.6450 - val_loss: 0.9412 - val_acc: 0.6930
Epoch 3/4
1000/1000 [==============================] - 55s - loss: 0.7232 - acc: 0.7860 - val_loss: 0.6118 - val_acc: 0.8180
Epoch 4/4
 512/1000 [==============>...............] - ETA: 21s - loss: 0.6679 - acc: 0.7695

#### Shearing

In [9]:
gen_t = image.ImageDataGenerator(shear_range=0.1, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [ ]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 68s - loss: 2.4955 - acc: 0.2280 - val_loss: 3.2073 - val_acc: 0.1690
Epoch 2/2
1000/1000 [==============================] - 54s - loss: 1.5611 - acc: 0.4870 - val_loss: 2.5235 - val_acc: 0.3040
Epoch 1/4
1000/1000 [==============================] - 55s - loss: 1.8570 - acc: 0.4140 - val_loss: 1.3541 - val_acc: 0.5740
Epoch 2/4
1000/1000 [==============================] - 55s - loss: 1.0812 - acc: 0.6450 - val_loss: 0.9412 - val_acc: 0.6930
Epoch 3/4
1000/1000 [==============================] - 55s - loss: 0.7232 - acc: 0.7860 - val_loss: 0.6118 - val_acc: 0.8180
Epoch 4/4
 512/1000 [==============>...............] - ETA: 21s - loss: 0.6679 - acc: 0.7695

#### Zoom

In [9]:
gen_t = image.ImageDataGenerator(zoom_range=0.1, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [ ]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 68s - loss: 2.4955 - acc: 0.2280 - val_loss: 3.2073 - val_acc: 0.1690
Epoch 2/2
1000/1000 [==============================] - 54s - loss: 1.5611 - acc: 0.4870 - val_loss: 2.5235 - val_acc: 0.3040
Epoch 1/4
1000/1000 [==============================] - 55s - loss: 1.8570 - acc: 0.4140 - val_loss: 1.3541 - val_acc: 0.5740
Epoch 2/4
1000/1000 [==============================] - 55s - loss: 1.0812 - acc: 0.6450 - val_loss: 0.9412 - val_acc: 0.6930
Epoch 3/4
1000/1000 [==============================] - 55s - loss: 0.7232 - acc: 0.7860 - val_loss: 0.6118 - val_acc: 0.8180
Epoch 4/4
 512/1000 [==============>...............] - ETA: 21s - loss: 0.6679 - acc: 0.7695

#### Channel Shift

In [9]:
gen_t = image.ImageDataGenerator(channel_shift_range=10, fill_mode='constant')
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [ ]:
model = test_model_conv(batches_train)

Epoch 1/2
1000/1000 [==============================] - 68s - loss: 2.4955 - acc: 0.2280 - val_loss: 3.2073 - val_acc: 0.1690
Epoch 2/2
1000/1000 [==============================] - 54s - loss: 1.5611 - acc: 0.4870 - val_loss: 2.5235 - val_acc: 0.3040
Epoch 1/4
1000/1000 [==============================] - 55s - loss: 1.8570 - acc: 0.4140 - val_loss: 1.3541 - val_acc: 0.5740
Epoch 2/4
1000/1000 [==============================] - 55s - loss: 1.0812 - acc: 0.6450 - val_loss: 0.9412 - val_acc: 0.6930
Epoch 3/4
1000/1000 [==============================] - 55s - loss: 0.7232 - acc: 0.7860 - val_loss: 0.6118 - val_acc: 0.8180
Epoch 4/4
 512/1000 [==============>...............] - ETA: 21s - loss: 0.6679 - acc: 0.7695

#### All

In [11]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1,
                                 rotation_range=15, shear_range=0.1,
                                 zoom_range=0.1, channel_shift_range=10,
                                 horizontal_flip=True)
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1000 images belonging to 10 classes.


In [14]:
model_conv = gen_model_conv(lr=0.001)
model_conv.fit_generator(batches_train, batches_train.n, nb_epoch=4,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n)

Epoch 1/4
1000/1000 [==============================] - 56s - loss: 2.5873 - acc: 0.1520 - val_loss: 2.1937 - val_acc: 0.1230
Epoch 2/4
1000/1000 [==============================] - 59s - loss: 2.2352 - acc: 0.2240 - val_loss: 2.0247 - val_acc: 0.2690
Epoch 3/4
1000/1000 [==============================] - 66s - loss: 2.1095 - acc: 0.2590 - val_loss: 2.1535 - val_acc: 0.2390
Epoch 4/4
1000/1000 [==============================] - 76s - loss: 2.0648 - acc: 0.2690 - val_loss: 1.8062 - val_acc: 0.3400


In [15]:
model_conv.fit_generator(batches_train, batches_train.n, nb_epoch=20,
                         validation_data=batches_valid, nb_val_samples=batches_valid.n)

Epoch 1/20
1000/1000 [==============================] - 58s - loss: 2.0057 - acc: 0.2810 - val_loss: 1.6103 - val_acc: 0.4020
Epoch 2/20
1000/1000 [==============================] - 61s - loss: 1.9216 - acc: 0.3340 - val_loss: 1.5911 - val_acc: 0.4570
Epoch 3/20
1000/1000 [==============================] - 63s - loss: 1.8867 - acc: 0.3430 - val_loss: 1.4882 - val_acc: 0.4620
Epoch 4/20
1000/1000 [==============================] - 71s - loss: 1.8229 - acc: 0.3380 - val_loss: 1.3811 - val_acc: 0.5660
Epoch 5/20
1000/1000 [==============================] - 69s - loss: 1.7744 - acc: 0.3850 - val_loss: 1.4230 - val_acc: 0.5040
Epoch 6/20
1000/1000 [==============================] - 85s - loss: 1.6492 - acc: 0.4280 - val_loss: 1.2146 - val_acc: 0.6100
Epoch 7/20
1000/1000 [==============================] - 78s - loss: 1.6039 - acc: 0.4670 - val_loss: 1.1731 - val_acc: 0.6010
Epoch 8/20
1000/1000 [==============================] - 75s - loss: 1.5430 - acc: 0.4730 - val_loss: 1.0914 - val_acc:

## Four Conv + Dropout

In [10]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1,
                                 rotation_range=15, shear_range=0.1,
                                 zoom_range=0.1, channel_shift_range=10)
gen = image.ImageDataGenerator()
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size)
batches_valid = gen.flow(data_valid, labels_valid, batch_size=batch_size, shuffle=False)

Found 17934 images belonging to 10 classes.


In [11]:
model = models.Sequential([
        BatchNormalization(axis=3, input_shape=(224, 224, 3)),
        Convolution2D(32, 3, 3, activation='relu'),
        BatchNormalization(axis=3),
        MaxPooling2D((2, 2)),
        Convolution2D(64, 3, 3, activation='relu'),
        BatchNormalization(axis=3),
        MaxPooling2D((2, 2)),
        Convolution2D(128, 3, 3, activation='relu'),
        BatchNormalization(axis=3),
        MaxPooling2D((2, 2)),       
        Flatten(),
        MaxoutDense(256),
        BatchNormalization(),
        Dropout(0.0),
        MaxoutDense(256),
        BatchNormalization(),
        Dropout(0.0),
        Dense(10, activation='softmax')        
    ])

In [12]:
model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(batches_train, batches_train.n, nb_epoch=2,
                   validation_data=batches_valid, nb_val_samples=batches_valid.n)

Epoch 1/2
17934/17934 [==============================] - 319s - loss: 1.5155 - acc: 0.4900 - val_loss: 2.7036 - val_acc: 0.2815

In [14]:
K.set_value(model.optimizer.lr, 0.0001)

In [15]:
model.fit_generator(batches_train, batches_train.n, nb_epoch=2,
                   validation_data=batches_valid, nb_val_samples=batches_valid.n)

Epoch 1/2
17934/17934 [==============================] - 294s - loss: 0.3556 - acc: 0.8856 - val_loss: 0.1620 - val_acc: 0.9548

In [16]:
K.set_value(model.optimizer.lr, 0.0001)

In [17]:
callbacks = [ModelCheckpoint(
        path_checkpoint+'weights_4conv.{epoch:02d}-{val_loss:.3f}-{val_acc:.3f}.h5',
        save_best_only=False, monitor='val_acc')]
model.fit_generator(batches_train, batches_train.n, nb_epoch=4,
                   validation_data=batches_valid, nb_val_samples=batches_valid.n,
                   callbacks=callbacks)

Epoch 1/4
17934/17934 [==============================] - 292s - loss: 0.2364 - acc: 0.9292 - val_loss: 0.1099 - val_acc: 0.9713

#### Checkpoint

In [9]:
model.load_weights(glob(path_checkpoint+'weights_4conv.*')[0])

IndexError: list index out of range

#### Submit

In [25]:
batches_test = utils.get_batches(path+'test', batch_size=batch_size, shuffle=False,
                                 class_mode=None)
preds = model.predict_generator(batches_test, batches_test.n)
np.save(path_results+'filenames_test_4conv.npy', batches_test.filenames)
np.save(path_results+'probs_test_4conv.npy', preds)

In [33]:
submit = pd.DataFrame(preds, columns=['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
submit['img'] = [x.split('/')[-1].split('.')[-2] for x in batches_test.filenames]
submit.to_csv(path_submit+'conv4.csv', index=False)
FileLink(path_submit+'conv4.csv')

/home/ubuntu/projects/fast_ai/data/statefarm/models/submissions/conv4.csv

# VGG19

In [3]:
import vgg19; importlib.reload(vgg19)
from vgg19 import Vgg19

In [4]:
vgg = Vgg19()

Loading model weights


## Conv Features

##### Create conv model

In [5]:
layers_conv, layers_fc = utils.split_at(vgg.model, Conv2D)

In [6]:
vgg_conv = models.Sequential(layers_conv)

##### Precalculate conv features

In [ ]:
feats_conv_train = vgg_conv.predict(batches_train, batches_train.n)
feats_conv_valid = vgg_conv.predict(batches_valid, batches_valid.n)
# feats_conv_test = vgg_conv.predict(batches_test, batches_test.n)
# feats_conv_train = vgg_conv.predict_generator(batches_train, batches_train.n)
# feats_conv_valid = vgg_conv.predict_generator(batches_valid, batches_valid.n)
feats_conv_test = vgg_conv.predict_generator(batches_test, batches_test.n)

In [ ]:
utils.save_array_bcolz(path_img_arrays+'train_conv.b5', feats_conv_train)
utils.save_array_bcolz(path_img_arrays+'valid_conv.b5', feats_conv_valid)
# utils.save_array_bcolz(path_img_arrays+'test_conv.b5', feats_conv_test)

In [ ]:
feats_conv_train = utils.load_array_bcolz(path_img_arrays+'train_conv.b5')
feats_conv_valid = utils.load_array_bcolz(path_img_arrays+'valid_conv.b5')
# feats_conv_test = utils.load_array_bcolz(path_img_arrays+'test_conv.b5')

##### Precalculate data augmented conv features

In [ ]:
gen_t = image.ImageDataGenerator(width_shift_range=0.15, height_shift_range=0.1,
                                 rotation_range=15, shear_range=0.1,
                                 zoom_range=0.1, channel_shift_range=20)
gen = image.ImageDataGenerator()
batches_train = utils.get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

In [ ]:
feats_conv_train_da = vgg_conv.predict_generator(batches_train, batches_train.n)

## Simplified fc layers

In [8]:
def gen_fcbn_layers(p):
    return[
        MaxPooling2D(input_shape=layers_conv[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        MaxoutDense(128),
        BatchNormalization(),
        Dropout(p/2),
        MaxoutDense(128),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
    ]
vgg_fc = models.Sequential(gen_fcbn_layers(0.8))
vgg_fc.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [ModelCheckpoint(
        path_checkpoint+'weights_4conv.{epoch:02d}-{val_loss:.3f}-{val_acc:.3f}.h5',
        save_best_only=False, monitor='val_acc')]
vgg_fc.fit(feats_conv_train, labels_train, batch_size=batch_size, nb_epoch=1,
           validation_data=(feats_conv_valid, labels_valid), callbacks=callbacks)